In [1]:
import os
import numpy as np
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
from shapely.geometry import Point
import pickle
import time
import datetime

# First define some functions

In [4]:
# Find hrefs in main search page

def generate_links(response):
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Index from 3 in order to skip promoted offers
    offer_links = []
    for offer in soup.find_all(class_='offer-item-details')[3:]:
        offer_links.append(offer.find('a').attrs['href'])
        
    return offer_links

In [5]:
# Find data in single offer's page and return as a json

def process_offer_link(link):
    
    # Get full html page
    offer_response = requests.get(link)
    # Convert to soup
    offer_soup = BeautifulSoup(offer_response.text, 'html.parser')
    
    # Find interesting data
    apartament_data = json.loads(offer_soup.find(id="server-app-state").text)['initialProps']['data']['advert']
    
    return apartament_data

In [6]:
# Function for extracting relevant data from json

def collect_data_from_json(apartament_data):
    
    # Uncomment below to review json
    #print(json.dumps(apartament_data, indent=4, sort_keys=True)) 
    
    # Initiate empty dict
    parsed_data = dict()
    
    # Price
    parsed_data['Cena'] = apartament_data['price']['value']
    
    # Basic infos
    for factor in apartament_data['characteristics']:
        if factor['label'] in ['Czynsz - dodatkowo', 'Kaucja', 'Powierzchnia']:
            parsed_data[factor['label']] = factor['value']
        else:
            parsed_data[factor['label']] = factor['value_translated']
    
    # Date of last edit >>personal data<<
    #parsed_data['Data ogłoszenia'] = apartament_data['dateModified']
    
    # Adres >>personal data<<
    #parsed_data['Adres'] = apartament_data['location']['address']
    
    # URL >>personal data<<
    #parsed_data['URL'] = apartament_data['url']
    
    # Geolocation
    latitude = apartament_data['location']['coordinates']['latitude']
    longitude = apartament_data['location']['coordinates']['longitude']
    radius = apartament_data['location']['coordinates']['radius']
    (parsed_data['φ'], parsed_data['λ']) = (latitude, longitude) if radius == 0 else (np.nan, np.nan)
    
    # Links to small photos >>personal data<<
    photos = [i['thumbnail'] for i in apartament_data['photos']]
    
    return parsed_data, photos

# Now lets get to scrap!

In [30]:
# Load 90 pages each with 72 results of apartaments

offer_links = []

for page in range(90):
    response = requests.get('https://www.otodom.pl/wynajem/mieszkanie/warszawa/?search%5Bdescription%5D=1&search%5Bcity_id%5D=26&nrAdsPerPage=72'
                       + '&page={}'.format(page+1))
    
    offer_links += generate_links(response)
    print("Processed page number {}".format(page+1))

Processed page number 1
Processed page number 2
Processed page number 3
Processed page number 4
Processed page number 5
Processed page number 6
Processed page number 7
Processed page number 8
Processed page number 9
Processed page number 10
Processed page number 11
Processed page number 12
Processed page number 13
Processed page number 14
Processed page number 15
Processed page number 16
Processed page number 17
Processed page number 18
Processed page number 19
Processed page number 20
Processed page number 21
Processed page number 22
Processed page number 23
Processed page number 24
Processed page number 25
Processed page number 26
Processed page number 27
Processed page number 28
Processed page number 29
Processed page number 30
Processed page number 31
Processed page number 32
Processed page number 33
Processed page number 34
Processed page number 35
Processed page number 36
Processed page number 37
Processed page number 38
Processed page number 39
Processed page number 40
Processed

In [ ]:
# In order to not overload the server, we can wait 1 sec before processing the a link

apartaments_data = []
for i, link in enumerate(offer_links[:3600]):
    try:
        apartaments_data.append(process_offer_link(link))
        print("Processed link number {}".format(i+1))
    except:
        print("Some error happened while processing link number {}".format(i+1))
    time.sleep(1)

# Dump `apartaments_data` (json) and `offer_links` (list) to pickles for backups!

In [43]:
if not os.path.isdir("pickles"):
     os.mkdir("pickles")

with open('pickles/apartaments_data.pkl', 'wb') as f:
    pickle.dump(apartaments_data, f)
with open('pickles/offer_links.pkl', 'wb') as f:
    pickle.dump(offer_links, f)

# (OR) Load pickle files for `apartaments_data` and `offer_links`

In [74]:
with open('pickles/apartaments_data.pkl', 'rb') as f:
    apartaments_data = pickle.load(f) 
with open('pickles/offer_links.pkl', 'rb') as f:
    offer_links = pickle.load(f)

# Extract relevant features from json

In [75]:
df_list = []
photos_list = []
for house_data in apartaments_data:
    
    parsed_data, photos = collect_data_from_json(house_data)
    
    photos_list.append(photos)

    # Append features data to the list of DataFrames
    columns = list(parsed_data.keys())
    rows = list(parsed_data.values())
    df_list.append(pd.DataFrame([rows], columns=columns))

In [76]:
# Concatonate all DataFrames in desired way

df = pd.concat(df_list, sort=False).reset_index(drop=True)
df = df.drop(columns = 'Dostępne od')  # drop because it's a personal data

In [77]:
# Add photos_list (as columns) to the DataFrame

"""if 'Zdjęcia' in df:
    df = df.drop(columns = 'Zdjęcia')
df.insert(df.shape[1], "Zdjęcia", str(photos_list).strip('[]'))"""
# Skip because it's a personal data

'if \'Zdjęcia\' in df:\n    df = df.drop(columns = \'Zdjęcia\')\ndf.insert(df.shape[1], "Zdjęcia", str(photos_list).strip(\'[]\'))'

# Remove duplicated advertisements

In [78]:
print(len(df), len(df.drop_duplicates()))
df = df.drop_duplicates()

3564 3525


# Save as .csv for future analyse

In [79]:
df.to_csv("apartaments.csv", index=False)

In [80]:
df.head()

,Cena,Kaucja,Powierzchnia,Liczba pokoi,Rodzaj zabudowy,Piętro,Okna,Ogrzewanie,Stan wykończenia,φ,λ,Czynsz - dodatkowo,Liczba pięter,Materiał budynku,Rok budowy
0,2400,2400,48,3,blok,2,plastikowe,miejskie,do zamieszkania,52.231618,21.066343,NaN,NaN,NaN,NaN
1,2400,3000,45,2,apartamentowiec,2,plastikowe,miejskie,do zamieszkania,52.289194,20.930776,500,5,cegła,2014
2,2600,NaN,40,2,apartamentowiec,4,plastikowe,miejskie,do zamieszkania,52.199425,21.044630,NaN,6,pustak,2018
3,2300,2500,54,2,blok,1,plastikowe,miejskie,do zamieszkania,52.287912,21.054128,400,5,NaN,2007
4,2950,NaN,54,3,blok,3,drewniane,miejskie,do zamieszkania,52.153523,21.082662,350,4,cegła,2016
